**Parsing Notes**

Nothing to extract for now: pull data into a dict
{ 'text' : .... }


In [30]:
from bs4 import BeautifulSoup as Soup
from IPython.display import display, HTML
import pandas as pd
import re
import pprint
import json

In [31]:
fn = 'procPublicationRequest Oct-Dec 2014 (Updated) - Sheet1-2.csv'
rows = pd.read_csv(fn, header=0)
cols = ['RequestID', 'StartDate', 
        'EndDate', 'AgencyCode', 'AgencyName', 
        'AgencyDivision', 'TypeOfNoticeCode', 
        'TypeOfNoticeDescription', 'ShortTitle', 
        'SectionID', 'SectionName', 'DueDate', 
        'ConfirmationNumber', 'AdditionalDescription', 'Address1']
rows = rows[cols]

In [32]:
def scrape(row):
    output = {}
    if row['AgencyName'] == "Mayor's Office of Contract Services" and \
       row['TypeOfNoticeDescription'] == "Meeting":
        try:
            output['text'] = Soup(row.AdditionalDescription).get_text(strip=True)
        except Exception:
            output = { 'error' : 'bad input', 'text': text }
        row['output'] = json.dumps(output)
    return row

In [33]:
fix = rows.apply(scrape,1)
mocs = fix['AgencyName'] == "Mayor's Office of Contract Services"
meets = fix['TypeOfNoticeDescription'] == "Meeting"
fix = fix[meets & mocs]


In [34]:
for rec in fix['output'].values:
    pprint.pprint(json.loads(rec))
    print('\n\n')

{'text': 'PUBLIC NOTICE IS HEREBY GIVEN THATtheFranchise and Concession '
         'Review Committee will hold aPublic Meeting on Wednesday, October '
         '8, 2014 at 2:30 p.m.,at 22 Reade Street, Spector Hall, Borough of '
         'Manhattan.NOTE: Individuals requesting Sign Language '
         "Interpretersshould contact the Mayor's Office of Contracts "
         'Services,Public Hearings Unit, 253 Broadway, 9th Floor,New York, '
         'NY 10007 (212) 788-7490, no laterthan SEVEN (7) BUSINESS DAYS '
         'PRIOR TO THE PUBLIC MEETING. TDDusers should call Verizon relay '
         'service.'}



{'text': 'PUBLIC NOTICE IS HEREBY GIVEN THATtheFranchise and Concession '
         'Review Committee will hold aPublic Meeting on Wednesday'}



{'text': 'PUBLIC NOTICE IS HEREBY GIVEN THATtheFranchise and Concession '
         'Review Committee will hold aPublic Meeting on Wednesday, December '
         '10, 2014 at 2:30 p.m.,at 22 Reade Street, Spector Hall, Borough of '
        